In [2]:
import numpy as np
import pandas as pd

In [13]:
# NaN is a numpy value for not a number and will cause calculations to not be doable. We will want to filter these out or fix them
# can filter them out with ~np.isnan, or np.isfinite which will also detect np.inf
example = np.array([1, 2, 3, np.nan, np.nan, 4])
example[~np.isnan(example)]


array([1., 2., 3., 4.])

In [6]:
# We can also use pandas it detect null values with isnull or isna
pd.isnull(np.nan)

True

In [15]:
# This can also work on Series and dataframes!
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


In [18]:
# Pandas will ignore null items when doing calculations, unlike numpy
pd.Series([1, 2, np.nan]).count()

2

In [19]:
# if we want to simply get rid of any null values we can use pandas to drop them
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])
pd.isnull(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [20]:
# This will display the not null values for the series
s[s.notnull()]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

In [21]:
# This will drop all null values, but s will still have them, this just displays it, do s = s.dropna() to actually change s
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

In [24]:
null_df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [26]:
# when using info on a dataframe can give you information on how many nulls there are, can be helpful if also looking at the other information given too
null_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 256.0 bytes


In [35]:
# or you cna just count up the null values if only looking for nulls
null_df.isnull().sum()


Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

In [37]:
# can also do a boolean check for the len between the normal series and the one without nulls
missing_values = len(s.dropna()) != len(s)
missing_values

True

In [28]:
# dropna on dataframes will cause any row with a null to be deleted by default
# once again, this is not actually changing the table and need to do null_df = null_df.dropna() to change table
null_df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can change this dropna for dataframes to be columns with the axis parameter.  
Futhermore, you can set the how parameter to be any or all for controlling if the whole row/column needs to be null  
Another option is thresh for setting a threshold amount for how many nulls are needed for the row/col be dropped  

Even with these parameters it can still be a very drastic action to remove whole rows and collumns depending on the dataset.  

Another option is to fill in the null values, but this also needs information on the dataset, and what the use of the data will be, to figure out if this is even a viable course of action at all.

Some options are filling them in with a specified value like 0, filling them in with values close to the ones they are nearby, filling them in with the mean of the dataset, or using ffill and bfill to front or back fill in the values into the nulls. Depening on the use case these might be viable options.

These are all immutable methods too, so you need to set the series/dataframe equal to the 

In [30]:
# filled with 0
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

Notice how the extremity nulls are not caught using the ffill and bfill

In [33]:
null_df.fillna(method='ffill', axis=0)


,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [34]:
null_df.fillna(method='ffill', axis=1)


,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


## Invalid Values  

Sometimes a dataset will have values that simply do not make sense, like this dataframe below with an age that is 290 and confusing inputs for sex (in a simplified dataset of just M/F for this example). 

This dataset is small enough to just see the values, but in larger datasets how would you be able to identify them? 

One way can be using the value count so see how many times a value is used in the data, this is helpful if the data has an expected domain to be in that is not incredibely large.

In [39]:
ages_df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25],
})

In [40]:
ages_df['Sex'].value_counts()

F    2
M    1
D    1
?    1
Name: Sex, dtype: int64

In [41]:
# we can use replace with either just 1 value to be replaced, or with a dictionary to replace mistakes.
# in this dictionary example we are replacing N with M as that is a common typo although there are no N's in this dataframe so just as an example.
ages_df['Sex'].replace({'D': 'F', 'N': 'M'})


0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

NOTE: This is immutable method too so the ages_df dataframe has not actually changed yet, and you would need to set it equal to the result. 

In [44]:
# you can do the same action on multiple 
ages_df.replace({
    'Sex': {
        'D': 'F',
        'N': 'M',
        '?': 'M'
    },
    'Age': {
        290: 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,M,25


In [47]:
# searching for duplicates is also easy, say you can only invite one ambassador from each country:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [48]:
# the keep parameter lets us not exclude the first or last duplicate
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [49]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [50]:
# for drop duplicates you can also use the keep parameter to change if you keep the first, last, or none of the duplicates.
ambassadors.drop_duplicates(keep='last')

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [51]:
# by default for dataframes, duplicate looks for the whole row to be duplicated
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})
# notice how Kobe is only duplicated once because the last time he has a different position.
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

In [52]:
# we can use the subset to look at individual values and not just the whole row, and the keep parameter works the same with dataframes
# by default keep = first
players.duplicated(subset=['Name'])


0    False
1    False
2     True
3    False
4     True
dtype: bool

## Text data 

Text data can be complicated as there is not really a set way of having text in data and it can change, furthermore a common problem is a mistype that is just human error when typing. 

Here is an example of a dataframe with input from a survey asking for birth year, sex, location, and number of children:

In [59]:
survey_df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})
survey_df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


In [54]:
# luckily we know that the different categories are seperated by _
survey_df['Data'].str.split('_')


0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [60]:
# expand will have it create a dataframe with new columns. We need to set the dataframe equal to actually make the change like we see here.
survey_df = survey_df['Data'].str.split('_', expand=True)
# furthermore we can set the column names to have an actual data set that represents the survey conducted.
survey_df.columns = ['Year', 'Sex', 'Country', 'No Children']
survey_df


,Year,Sex,Country,No Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2
